# Machine Learning Regression Problem 

## Predicting Price of a House


# Using Pytorch

In [ ]:
"""
step 1. Pre-process:
        Find the 'numeric' data and process (mean, std) the 'numeric' data
        Complete the 'object' data without Nan

step 2. Prepare dataset converting to the tensor type

step 3. Create model

step 4. Train model

step 5. Predict the test.csv, meanwhile, upload the result.csv
"""

In [1]:
import torch.nn as nn
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import timeit

In [175]:
#df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
#df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
melbourne_data =  pd.read_csv("/home/gitika/My_Home/Python_ML_CNN/Regression/Melbourne_housing_FULL_with_missing.csv")
print(melbourne_data.describe())



              Rooms         Price      Distance      Postcode      Bedroom2  \
count  34853.000000  2.724700e+04  34856.000000  34856.000000  26640.000000   
mean       3.031045  1.050173e+06     11.184929   3116.062859      3.084647   
std        0.969943  6.414671e+05      6.788892    109.023903      0.980690   
min        1.000000  8.500000e+04      0.000000   3000.000000      0.000000   
25%        2.000000  6.350000e+05      6.400000   3051.000000      2.000000   
50%        3.000000  8.700000e+05     10.300000   3103.000000      3.000000   
75%        4.000000  1.295000e+06     14.000000   3156.000000      4.000000   
max       16.000000  1.120000e+07     48.100000   3978.000000     30.000000   

           Bathroom           Car       Landsize  BuildingArea     YearBuilt  \
count  26631.000000  26129.000000   23047.000000   13742.00000  15551.000000   
mean       1.624798      1.728845     593.598993     160.25640   1965.289885   
std        0.724212      1.010771    3398.841946

In [223]:
from sklearn.model_selection import train_test_split

print(melbourne_data.columns)
y = melbourne_data.Price
X = melbourne_data.drop('Price',axis=1)

#Split the data into training set and testing set and then use created csv files
df_train_X, df_test_X, df_train_y, df_test_y = train_test_split(X, y, train_size=0.6,random_state=10)

df_train = pd.concat([df_train_X,df_train_y],axis=1)
df_test = pd.concat([df_test_X,df_test_y],axis=1)

df_train.to_csv("./Melbourne_housing_FULL_train.csv", columns=df_train.columns, index=False)
df_test.to_csv("./Melbourne_housing_FULL_test.csv", columns=df_test.columns, index=False)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')
7837      730000.0
18042    1705000.0
4834           NaN
31707     723000.0
28747          NaN
           ...    
10201    1211000.0
9372     2667000.0
28017     670000.0
29199    1200000.0
17673     850000.0
Name: Price, Length: 20914, dtype: float64


In [3]:
df_train = pd.read_csv("Melbourne_housing_FULL_train.csv")
df_test = pd.read_csv("Melbourne_housing_FULL_test.csv")
#print(df_train.describe())
print("df_train shape:", df_train.shape)
print("df_test shape:", df_test.shape)
means = df_train["Price"].mean()
maxs = df_train["Price"].max()
mins = df_train["Price"].min()

df_all = pd.concat([df_train, df_test], axis=0)
print("df_all shape:",df_all.shape)


df_train shape: (20914, 21)
df_test shape: (13943, 21)
df_all shape: (34857, 21)


In [7]:
### step 1 
# ！object

t_0 = timeit.default_timer()
print('Start time (t_0):',t_0)


numeric_features_index = df_all.dtypes[df_all.dtypes != 'object'].index
label_features_index = numeric_features_index[-1]
numeric_features_index = numeric_features_index[:-1]
df_all = df_all.drop(label_features_index, axis=1)

object_feature_index = df_all.dtypes[df_all.dtypes == 'object'].index

print("the lable name:", label_features_index)
print("the shape of numeric dtype indexs:",numeric_features_index.shape)
print("the shape of object dtype indexs:",object_feature_index.shape)

df_all[numeric_features_index] = df_all[numeric_features_index].apply(
    lambda x: (x - x.mean()) / (x.std()))

df_all[numeric_features_index] = df_all[numeric_features_index].fillna(0)

if np.all(df_all[numeric_features_index].isnull()):
    print("still survive the null")
    


# # object
new_all = pd.get_dummies(df_all, dummy_na=True)
print("the shape of preprocessed train data:",new_all.shape)
if np.all(new_all.isnull()):
    print("still survive the null")
    
input_size = new_all.shape[1]
print(f"the linear input size is {input_size}")
label_train = df_train[label_features_index]
print("the shape of label" ,label_train.shape)


t_f = timeit.default_timer() - t_0
print('Elapsed time (t_f):',t_f)

print(df_train.shape, new_all.shape)
print(df_train.shape[0], new_all[:20914])

Start time (t_0): 53356.237989475
the lable name: Propertycount
the shape of numeric dtype indexs: (11,)
the shape of object dtype indexs: (8,)
the shape of preprocessed train data: (34857, 34898)
the linear input size is 34898
the shape of label (20914,)
20914 (34857, 34898)
Elapsed time (t_f): 2.1621568920018035


**train: data(79) + label(1), total 80 columns**

**test: data(79) + label(1) but the lable is Nan**

In [ ]:
# step 2
num_train = df_train.shape[0]

train_features = torch.tensor((new_all[:num_train].values), dtype=torch.float32)

train_labels = torch.tensor(label_train.values.reshape(-1,1), 
                            dtype=torch.float32)

train_labels = (train_labels - means) / (maxs - mins)

dataset = torch.utils.data.TensorDataset(train_features,train_labels)

temp_num = int(len(dataset) * 0.95)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [temp_num, len(dataset) - temp_num])
print("the train dataset has ",len(train_dataset))
print("the val dataset has ",len(val_dataset))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32)

In [185]:
### step 1 
# ！object
#Here label_features_index is the target feature

label_features_index = numeric_features_index_orig[1]
print("label_features_index: ",label_features_index)

df_all = df_all_orig.drop(label_features_index, axis=1)

numeric_features_index = df_all.dtypes[df_all.dtypes != 'object'].index
object_feature_index = df_all.dtypes[df_all.dtypes == 'object'].index

print("the lable name:", label_features_index)
print("the shape of numeric dtype indexs:",numeric_features_index.shape)
print("the shape of object dtype indexs:",object_feature_index.shape)

print("df_all[['Rooms','Distance']] old",df_all[['Rooms','Distance']][0:6])

df_all[numeric_features_index] = df_all[numeric_features_index].apply(lambda x: (x - x.mean()) / (x.std()))
df_all[numeric_features_index] = df_all[numeric_features_index].fillna(0)

"""
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
df_all[numeric_features_index] = pd.DataFrame(my_imputer.fit_transform(df_all[numeric_features_index]))
"""
print("df_all[['Rooms','Distance']] new",df_all[['Rooms','Distance']][0:6])

if np.all(df_all[numeric_features_index].isnull()):
    print("still survive the null")
    


# # object
new_all = pd.get_dummies(df_all, dummy_na=True)
print("the shape of preprocessed train data:",new_all.shape)
if np.all(new_all.isnull()):
    print("still survive the null")
    
input_size = new_all.shape[1]
print(f"the linear input size is {input_size}")
label_train = df_train[label_features_index]
print("the shape of label" ,label_train.shape)


label_features_index:  Price
the lable name: Price
the shape of numeric dtype indexs: (12,)
the shape of object dtype indexs: (8,)
df_all[['Rooms','Distance']] old    Rooms  Distance
0    NaN       2.5
1    NaN       2.5
2    2.0       2.5
3    NaN       2.5
4    3.0       2.5
5    3.0       2.5
df_all[['Rooms','Distance']] new       Rooms  Distance
0  0.000000 -1.279285
1  0.000000 -1.279285
2 -1.062995 -1.279285
3  0.000000 -1.279285
4 -0.032007 -1.279285
5 -0.032007 -1.279285
the shape of preprocessed train data: (34857, 34899)
the linear input size is 34899
the shape of label (34857,)


'\n'

In [121]:
a = pd.DataFrame([2.0,3.0,3.0,3.0,2.0,2.0,2.0,8.0])
print(a.mean(),a.std())
x = a.apply(lambda x: (x - x.mean()) / (x.std()))


0    3.125
dtype: float64 0    2.03101
dtype: float64


In [ ]:
# step 2
num_train = df_train.shape[0]

train_features = torch.tensor((new_all[:num_train].values), dtype=torch.float32)

train_labels = torch.tensor(label_train.values.reshape(-1,1), 
                            dtype=torch.float32)

train_labels = (train_labels - means) / (maxs - mins)

dataset = torch.utils.data.TensorDataset(train_features,train_labels)

temp_num = int(len(dataset) * 0.95)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [temp_num, len(dataset) - temp_num])
print("the train dataset has ",len(train_dataset))
print("the val dataset has ",len(val_dataset))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32)

In [ ]:
# step 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class HousePricesPredict(nn.Module):
    def __init__(self, input_channels):
        super(HousePricesPredict, self).__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(input_channels, 1024),
            nn.Tanh(),
            nn.Linear(1024, 4096),
            nn.Tanh(),
            nn.Linear(4096, 4096),
            nn.Tanh(),
            nn.Linear(4096, 1024),
            nn.Tanh(),
            nn.Linear(1024, 1024),
            nn.Tanh(),
            nn.Linear(1024, 1),
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

model = HousePricesPredict(input_size)
model = model.to(device)
print(model)
    

In [ ]:
# step 4
loss_fn = nn.MSELoss()
# 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
train_loss_all = []
# 300
for epoch in range(100):
    train_loss = 0
    train_num = 0
    for step,(X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
#         print(X)
#         print(y)
        output = model(X)
#         print(output)
        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X.size(0)
        train_num += X.size(0)
#         print(train_loss)
    train_loss_all.append(train_loss / train_num)
# print(train_loss_all)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(train_loss_all, "ro-", label="Train loss")
plt.legend()
plt.grid()
plt.xlabel("epoch")
plt.ylabel("loss")

plt.show()

In [ ]:
length = len(val_dataset)
mae = 0.0
for step,(X, y) in enumerate(val_dataset):
    X, y = X.to(device), y.to(device)
    output = model(X)
    output = output.cpu().data.numpy()
    y = y.cpu().data.numpy()
    mae += mean_absolute_error(output, y)
print(mae / length)

In [ ]:
test_dataset = torch.tensor((new_all[num_train:].values), dtype=torch.float32)
# print(test_dataset[0])
test_dataset = test_dataset.to(device)
# for index, X in enumerate(test_dataset):
output = model(test_dataset).to(device)
# print(output)
result = pd.DataFrame(output.cpu().data.numpy(), columns=['SalePrice'])

if np.any(result):
    result['SalePrice'] = result['SalePrice'].fillna(0)

result['SalePrice'] = result['SalePrice'] * (maxs - mins) + means
result['Id'] = np.array(result.index)
result['Id'] = result['Id'] + 1461
result = pd.DataFrame(result, columns=['Id', 'SalePrice'])
result

In [ ]:
result.to_csv("./result.csv", columns=['Id', 'SalePrice'], index=False)

# END of Code